In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"deekshitthvegi","key":"abec15f9d7106d9d6fa2577e4695b5b0"}'}

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)

In [ ]:
!cp kaggle.json /root/.kaggle/

In [ ]:
os.chmod('/root/.kaggle/kaggle.json', 600)

In [ ]:
!kaggle competitions download -c wikipedia-image-caption -f train-00000-of-00005.tsv

100% 5.02G/5.04G [00:30<00:00, 126MB/s]
100% 5.04G/5.04G [00:31<00:00, 175MB/s]


In [ ]:
!unzip train-00000-of-00005.tsv.zip

Archive:  train-00000-of-00005.tsv.zip
  inflating: train-00000-of-00005.tsv  


In [ ]:
!rm train-00000-of-00005.tsv.zip

In [ ]:
!kaggle competitions download -c wikipedia-image-caption -f train-00001-of-00005.tsv

100% 5.04G/5.04G [00:43<00:00, 209MB/s]
100% 5.04G/5.04G [00:43<00:00, 125MB/s]


In [ ]:
!unzip train-00001-of-00005.tsv.zip

Archive:  train-00001-of-00005.tsv.zip
  inflating: train-00001-of-00005.tsv  


In [ ]:
!rm train-00001-of-00005.tsv.zip

In [ ]:
import pandas as pd

train_data_0 = pd.read_csv('train-00000-of-00005.tsv', sep='\t')
train_data_1 = pd.read_csv('train-00001-of-00005.tsv', sep='\t')
train_data = pd.concat([train_data_0, train_data_1], ignore_index=True)

train_data.head()

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description,caption_title_and_reference_description
0,fr,https://fr.wikipedia.org/wiki/Pariser_Kanonen,https://upload.wikimedia.org/wikipedia/commons...,Pariser Kanonen,Bilan,Pariser Kanonen / Bilan,NaN,Français&#160;: Plaque apposée au n° 81 du bou...,NaN,image/jpeg,1279,1812,False,True,True,Les Pariser Kanonen sont sept pièces d’artille...,Le ou les canons restants sont démontés devant...,Pariser Kanonen [SEP]
1,tg,https://tg.wikipedia.org/wiki/Republic_P-43_La...,http://upload.wikimedia.org/wikipedia/commons/...,Republic P-43 Lancer,NaN,Republic P-43 Lancer,NaN,A Republic P-43 Lancer in flight over Esler Fi...,NaN,image/jpeg,306,591,True,False,False,Republic P-43 Lancer — як ҳавогарди сохтаи Rep...,Republic P-43 Lancer (англ. Republic P-43 Lanc...,Republic P-43 Lancer [SEP]
2,en,"https://en.wikipedia.org/wiki/Deer_Park,_Wisco...",https://upload.wikimedia.org/wikipedia/commons...,"Deer Park, Wisconsin",NaN,"Deer Park, Wisconsin",Downtown Deer Park,"English: Downtown Deer Park, Wisconsin on WIS46.",Downtown Deer Park,image/jpeg,1824,2736,True,True,True,"Deer Park is a village in St. Croix County, Wi...","Deer Park is a village in St. Croix County, Wi...","Deer Park, Wisconsin [SEP] Downtown Deer Park"
3,pt,https://pt.wikipedia.org/wiki/Vaux-l%C3%A8s-Mo...,https://upload.wikimedia.org/wikipedia/commons...,Vaux-lès-Mouzon,NaN,Vaux-lès-Mouzon,NaN,Français&#160;: Vaux-lès-Mouzon&#160;: le village,NaN,image/jpeg,1704,2272,True,False,False,Vaux-lès-Mouzon é uma comuna francesa na regiã...,Vaux-lès-Mouzon é uma comuna francesa na regiã...,Vaux-lès-Mouzon [SEP]
4,ne,https://ne.wikipedia.org/wiki/%E0%A4%95%E0%A5%...,https://upload.wikimedia.org/wikipedia/commons...,क्रिसमस टापु,NaN,क्रिसमस टापु,NaN,This is a locator map for Christmas Island I m...,NaN,image/png,165,248,True,False,False,क्रिसमस टापु हिन्द महासागरमा अवस्थित अष्ट्रेलि...,क्रिसमस टापु हिन्द महासागरमा अवस्थित अष्ट्रेलि...,क्रिसमस टापु [SEP]


In [ ]:
train_data.info()
# Check for any missing values
missing_values = train_data.isnull().sum()
print("Missing values in each column:\n", missing_values)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14822092 entries, 0 to 14822091
Data columns (total 19 columns):
 #   Column                                   Dtype 
---  ------                                   ----- 
 0   language                                 object
 1   page_url                                 object
 2   image_url                                object
 3   page_title                               object
 4   section_title                            object
 5   hierarchical_section_title               object
 6   caption_reference_description            object
 7   caption_attribution_description          object
 8   caption_alt_text_description             object
 9   mime_type                                object
 10  original_height                          int64 
 11  original_width                           int64 
 12  is_main_image                            bool  
 13  attribution_passes_lang_id               bool  
 14  page_changed_recently           

In [ ]:
# 1. Combine the caption-related columns to handle missing values.
train_data['caption'] = (
    train_data['caption_reference_description'].fillna('') + ' ' +
    train_data['caption_attribution_description'].fillna('') + ' ' +
    train_data['caption_alt_text_description'].fillna('') + ' ' +
    train_data['caption_title_and_reference_description'].fillna('')
).str.strip()

# 2. Drop rows where captions are still missing (completely empty after concatenation)
train_data = train_data[train_data['caption'] != '']
# 3. Filter the data to include only the main images
train_data_main = train_data[train_data['is_main_image']]
# 4. Keep only the necessary columns, now using `train_data_main`
train_data_cleaned = train_data_main[['image_url', 'caption']]
# 5. Optional: Save the processed data for future use
train_data_cleaned.to_csv('cleaned_train_data.csv', index=False)
train_data_cleaned.head()

,image_url,caption
1,http://upload.wikimedia.org/wikipedia/commons/...,A Republic P-43 Lancer in flight over Esler Fi...
2,https://upload.wikimedia.org/wikipedia/commons...,Downtown Deer Park English: Downtown Deer Park...
3,https://upload.wikimedia.org/wikipedia/commons...,Français&#160;: Vaux-lès-Mouzon&#160;: le vill...
4,https://upload.wikimedia.org/wikipedia/commons...,This is a locator map for Christmas Island I m...
6,https://upload.wikimedia.org/wikipedia/commons...,Čeština: Kříž v ohrádce u komunikace z Kamýku ...


In [ ]:
import re

# Function to clean captions
def clean_caption(text):
    # Remove HTML entities like &#160;, etc.
    text = re.sub(r'&#\d+;', ' ', text)
    # Remove any other HTML tags (like <br>)
    text = re.sub(r'<.*?>', '', text)
    # Remove excess whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean the captions in the DataFrame safely
train_data_cleaned.loc[:, 'caption'] = train_data_cleaned['caption'].apply(clean_caption)
train_data_cleaned.head()

,image_url,caption
1,http://upload.wikimedia.org/wikipedia/commons/...,A Republic P-43 Lancer in flight over Esler Fi...
2,https://upload.wikimedia.org/wikipedia/commons...,Downtown Deer Park English: Downtown Deer Park...
3,https://upload.wikimedia.org/wikipedia/commons...,Français : Vaux-lès-Mouzon : le village Vaux-l...
4,https://upload.wikimedia.org/wikipedia/commons...,This is a locator map for Christmas Island I m...
6,https://upload.wikimedia.org/wikipedia/commons...,Čeština: Kříž v ohrádce u komunikace z Kamýku ...


In [ ]:
print(train_data_cleaned['image_url'].head())

1    http://upload.wikimedia.org/wikipedia/commons/...
2    https://upload.wikimedia.org/wikipedia/commons...
3    https://upload.wikimedia.org/wikipedia/commons...
4    https://upload.wikimedia.org/wikipedia/commons...
6    https://upload.wikimedia.org/wikipedia/commons...
Name: image_url, dtype: object


In [ ]:
!rm -r dataset #this one is just to delete the imges directory. I'll remove it later

In [ ]:
import os
import pandas as pd
import requests
from tqdm import tqdm

def download_image(url, save_path):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} for URL: {url}")
        return False
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return False

image_dir = 'dataset/images'
os.makedirs(image_dir, exist_ok=True)

dataset = []
failed_downloads = []
total_failed = 0

for index, row in tqdm(train_data_cleaned.iterrows(), total=train_data_cleaned.shape[0]):
    image_url = row['image_url']
    caption = row['caption']

    image_file_name = f"image_{index}.jpg"
    image_save_path = os.path.join(image_dir, image_file_name)

    if os.path.exists(image_save_path):
        print(f"Image already exists: {image_save_path}. Skipping download.")
        dataset.append({'image_file': image_file_name, 'caption': caption})
        continue

    if not download_image(image_url, image_save_path):
        failed_downloads.append({'url': image_url, 'index': index})
        total_failed += 1

dataset_df = pd.DataFrame(dataset)
dataset_df.to_csv('dataset/captions.csv', index=False)

if failed_downloads:
    failed_df = pd.DataFrame(failed_downloads)
    failed_df.to_csv('dataset/failed_downloads.csv', index=False)

print(f"Dataset creation complete. Images and captions saved.")
print(f"Total failed downloads: {total_failed}")

  0%|          | 125/4909224 [00:27<266:13:39,  5.12it/s]

HTTP error occurred: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/5/55/Chiesa-quintano.jpg for URL: http://upload.wikimedia.org/wikipedia/commons/5/55/Chiesa-quintano.jpg


  0%|          | 154/4909224 [00:34<239:31:29,  5.69it/s]

HTTP error occurred: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/f/fe/Cyrus_portrait.jpg for URL: https://upload.wikimedia.org/wikipedia/commons/f/fe/Cyrus_portrait.jpg


  0%|          | 157/4909224 [00:35<269:47:45,  5.05it/s]

...

  0%|          | 310/4909224 [01:10<264:17:22,  5.16it/s]

HTTP error occurred: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/a/a7/Attiki_Metro_Evangelismos4.jpg for URL: https://upload.wikimedia.org/wikipedia/commons/a/a7/Attiki_Metro_Evangelismos4.jpg


  0%|          | 372/4909224 [01:24<205:54:48,  6.62it/s]

HTTP error occurred: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/2/21/Nte-kir-1bela.jpg for URL: https://upload.wikimedia.org/wikipedia/commons/2/21/Nte-kir-1bela.jpg
HTTP error occurred: 404 Client Error: Not Found for url: https://upload.wikimedia.org/wikipedia/commons/b/bc/Marcelo_Moura_en_2017.jpg for URL: https://upload.wikimedia.org/wikipedia/commons/b/bc/Marcelo_Moura_en_2017.jpg


  0%|          | 389/4909224 [01:28<313:31:07,  4.35it/s]

In [ ]:
num_image_urls = train_data['image_url'].nunique()
num_image_urls

6829977